# Sentinel-2 Download Example

Questo notebook dimostra come usare il sistema di download con OAuth2 e Sentinel Hub API.

## 1. Import dei Moduli

In [1]:
from satellite_analysis.config import Config
from satellite_analysis.downloaders.auth import OAuth2AuthStrategy
from satellite_analysis.downloaders.catalog import SentinelHubCatalog
import json

## 2. Caricamento Configurazione

La configurazione viene caricata da `config/config.yaml` e contiene:
- Credenziali OAuth2 (client_id, client_secret)
- Area di interesse (Milano)
- Parametri di ricerca (cloud cover, date)

In [2]:
# Carica config
config = Config.from_yaml("../config/config.yaml")

print(f"📍 Area: {config.area.city}, {config.area.country}")
print(f"🛰️  Platform: {config.sentinel.platformname}")
print(f"☁️  Max cloud: {config.sentinel.max_cloud_cover}%")
print(f"🔑 Client ID: {config.sentinel.client_id[:15]}...")

📍 Area: Milan, Italy
🛰️  Platform: Sentinel-2
☁️  Max cloud: 10.0%
🔑 Client ID: sh-f6fcded7-638...


## 3. Autenticazione OAuth2

**Flusso**:
```
OAuth2AuthStrategy.__init__(client_id, client_secret)
    ↓
OAuth2AuthStrategy.get_session()
    ↓
OAuth2AuthStrategy._authenticate()
    ↓
BackendApplicationClient (oauthlib)
    ↓
OAuth2Session.fetch_token()
    ↓
POST https://identity.dataspace.copernicus.eu/.../token
    ↓
Ritorna: OAuth2Session con token valido
```

In [3]:
# Crea strategia di autenticazione
auth = OAuth2AuthStrategy(
    client_id=config.sentinel.client_id,
    client_secret=config.sentinel.client_secret
)

# Ottieni sessione autenticata
session = auth.get_session()

print(f"✅ Autenticazione completata")
print(f"Token valido: {auth.is_valid()}")

✅ OAuth2 authentication successful!
✅ Autenticazione completata
Token valido: True


## 4. Ricerca nel Catalogo

**Flusso**:
```
SentinelHubCatalog.__init__(session)
    ↓
SentinelHubCatalog.search(bbox, dates, cloud_cover)
    ↓
SentinelHubCatalog._validate_bbox()
SentinelHubCatalog._validate_dates()
SentinelHubCatalog._validate_cloud_cover()
    ↓
POST https://sh.dataspace.copernicus.eu/api/v1/catalog/1.0.0/search
    ↓
Filtraggio client-side per cloud cover
    ↓
Ritorna: Dict con 'features' (lista di prodotti)
```

In [5]:
# Crea catalogo
catalog = SentinelHubCatalog(session)

# Definisci area di ricerca (Milano)
bbox = [9.0, 45.3, 9.3, 45.6]  # [min_lon, min_lat, max_lon, max_lat]
start_date = "2025-07-01"
end_date = "2025-08-01"

# Cerca prodotti
results = catalog.search(
    bbox=bbox,
    start_date=start_date,
    end_date=end_date,
    collection="sentinel-2-l2a",
    cloud_cover_max=config.sentinel.max_cloud_cover,
    limit=5
)

print(f"\n✅ Ricerca completata")
print(f"Prodotti trovati: {len(results['features'])}")

✅ Found 2 products (cloud cover <= 10.0%)

✅ Ricerca completata
Prodotti trovati: 2


## 5. Analisi Risultati

In [6]:
# Mostra dettagli prodotti
for idx, feature in enumerate(results['features'], 1):
    props = feature.get('properties', {})
    
    print(f"\n{'='*60}")
    print(f"Prodotto #{idx}")
    print(f"{'='*60}")
    print(f"ID: {props.get('id', 'N/A')}")
    print(f"Data: {props.get('datetime', 'N/A')}")
    print(f"Cloud Cover: {props.get('eo:cloud_cover', 'N/A')}%")
    print(f"Platform: {props.get('platform', 'N/A')}")
    print(f"Instrument: {props.get('instruments', 'N/A')}")
    
    # Geometry
    geom = feature.get('geometry', {})
    print(f"Geometry Type: {geom.get('type', 'N/A')}")
    
    # Assets (link ai dati)
    assets = feature.get('assets', {})
    print(f"\nAssets disponibili: {len(assets)}")
    for asset_name in list(assets.keys())[:5]:  # Primi 5
        print(f"  - {asset_name}")


Prodotto #1
ID: N/A
Data: 2025-07-29T10:28:18.458Z
Cloud Cover: 3.91%
Platform: sentinel-2b
Instrument: ['msi']
Geometry Type: MultiPolygon

Assets disponibili: 1
  - data

Prodotto #2
ID: N/A
Data: 2025-07-29T10:28:14.721Z
Cloud Cover: 7.18%
Platform: sentinel-2b
Instrument: ['msi']
Geometry Type: MultiPolygon

Assets disponibili: 1
  - data


## 6. Struttura Completa dei Risultati

In [7]:
# Visualizza struttura JSON completa del primo prodotto
if results['features']:
    print(json.dumps(results['features'][0], indent=2, default=str))

{
  "stac_version": "1.0.0",
  "stac_extensions": [
    "https://stac-extensions.github.io/eo/v1.0.0/schema.json",
    "https://stac-extensions.github.io/projection/v1.0.0/schema.json"
  ],
  "id": "S2B_MSIL2A_20250729T101559_N0511_R065_T32TMR_20250729T142051.SAFE",
  "type": "Feature",
  "geometry": {
    "type": "MultiPolygon",
    "crs": {
      "type": "name",
      "properties": {
        "name": "urn:ogc:def:crs:OGC::CRS84"
      }
    },
    "coordinates": [
      [
        [
          [
            7.7069642166980215,
            46.04625075575541
          ],
          [
            9.126154289789115,
            46.05349574829899
          ],
          [
            9.123961815688029,
            45.065201720695164
          ],
          [
            7.7294289213711265,
            45.05820080970157
          ],
          [
            7.7069642166980215,
            46.04625075575541
          ]
        ]
      ]
    ]
  },
  "bbox": [
    7.7069642166980215,
    45.0582008

## 7. Test Validazione Parametri

In [8]:
# Test 1: BBox invalida
try:
    catalog.search(
        bbox=[200, 45, 210, 46],  # lon > 180
        start_date=start_date,
        end_date=end_date,
        limit=1
    )
    print("❌ Doveva fallire!")
except ValueError as e:
    print(f"✅ BBox invalida correttamente rifiutata: {e}")

✅ BBox invalida correttamente rifiutata: Longitude must be between -180 and 180, got 200, 210


In [9]:
# Test 2: Date invertite
try:
    catalog.search(
        bbox=bbox,
        start_date="2023-05-01",
        end_date="2023-03-01",  # end < start
        limit=1
    )
    print("❌ Doveva fallire!")
except ValueError as e:
    print(f"✅ Date invertite correttamente rilevate: {e}")

✅ Date invertite correttamente rilevate: start_date must be before end_date: 2023-05-01 >= 2023-03-01


## 📊 Prossimi Step

1. **Download dei prodotti**: Implementare `download_product()` per scaricare i file
2. **Estrazione bande**: Estrarre specifiche bande dal file `.SAFE`
3. **Preprocessing**: Resampling e stacking delle bande
4. **Clustering**: Applicare gli algoritmi già implementati

### Struttura del Flusso Completo

```
Config.from_yaml()
    ↓
OAuth2AuthStrategy.get_session()
    ↓
SentinelHubCatalog.search()
    ↓
[TODO] Downloader.download_product()
    ↓
[TODO] Preprocessor.extract_bands()
    ↓
[TODO] Preprocessor.resample_bands()
    ↓
[DONE] ClusteringFactory.create('kmeans++')
    ↓
[DONE] clusterer.fit_predict(data)
```

In [ ]:
from satellite_analysis.downloaders import ProductDownloader

# Crea downloader
downloader = ProductDownloader(
    session=session,
    output_dir="data/raw"
)

# Download con controllo totale
if results['features']:
    # Filtra solo prodotti con cloud < 5%
    low_cloud_features = [
        f for f in results['features']
        if f['properties'].get('eo:cloud_cover', 100) < 5.0
    ]
    
    print(f"Prodotti con cloud < 5%: {len(low_cloud_features)}")
    
    # Download (commentato per non scaricare ora)
    # files = downloader.download_products(low_cloud_features, max_products=1)
    print("\n💡 Uncomment per scaricare effettivamente")

## 9. Download Manuale (Livello Intermedio)

Per maggior controllo, usa direttamente il `ProductDownloader`:

In [ ]:
from satellite_analysis.pipelines import DownloadPipeline

# Crea pipeline da config
pipeline = DownloadPipeline.from_config("../config/config.yaml")

# Esegui workflow completo
result = pipeline.run(
    bbox=bbox,
    start_date=start_date,
    end_date=end_date,
    limit=5,
    max_downloads=2  # Scarica solo i primi 2 (per test)
)

# Risultati
print(f"\n📊 SUMMARY:")
print(f"Total products found: {result.total_products}")
print(f"Downloaded: {result.downloaded_count}")
print(f"Failed: {result.failed_count}")
print(f"Success rate: {result.success_rate:.1f}%")

if result.downloaded_files:
    print(f"\n📁 Downloaded files:")
    for file_path in result.downloaded_files:
        print(f"  - {file_path}")

## 8. High-Level Pipeline (Livello Semplice)

Per utenti che vogliono un'interfaccia più semplice, usiamo la `DownloadPipeline`:

```python
from satellite_analysis.pipelines import DownloadPipeline

# Setup in 1 linea
pipeline = DownloadPipeline.from_config("../config/config.yaml")

# Esegui tutto (search + download)
result = pipeline.run(
    bbox=[9.0, 45.3, 9.3, 45.6],
    start_date="2023-03-01",
    end_date="2023-03-15",
    limit=5,
    max_downloads=2  # Scarica solo i primi 2
)

print(result)  # Summary automatico
```